In [1]:
using Optim
using DataFrames;using CSV
using Pkg
using LinearAlgebra;
using JuliaZH;JuliaZH.set_mirror("PKU")#JuliaZH.generate_startup("BFSU")
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)
using Base.Threads;nthreads()
using Meteor.ExactDiag;
using JLD2;using FileIO
using DifferentialEquations

In [2]:
function dm2vec(dm)
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,3,:)
end
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end
    
function coherence_state(alpha)
    m=exp(alpha*a₊(dim)-alpha'*a₋(dim))*n0(dim)
    return  m*m'
end
    
function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end
    
function equ_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    for i in 1:n
        c[i,i]=1/n    
    end
    return c
end
    
dim=4
η=0.084 
    
function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));
a=a₊(dim)+a₋(dim)    
a2=kron([0 0;1 0],exp(im*η*a))
a3=kron([0 1;0 0],exp(-im*η*a))
# a2=kron([0 0;1 0],im*η*a₊(dim))
# a3=kron([0 1;0 0],-im*η*a₋(dim))   
#经过尝试发现,符合脉冲并不精准,都保留下来就很不精确
    
σ₊=[0 0;1 0]
σ₋=[0 1;0 0]
σ_x=[0 1;1 0]
σ_y=[0 -1im;1im 0]
σ_z=[1 0;0 -1]

function dm2vec(dm)
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,Int(sqrt(size(vec)[1])),:)
end
    
function temperature(nbar)
    return (6.626*1e-34*1e6*1)/(1.38*1e-23*log(1/nbar+1))
end
    
n=kron(I(2),a₊(dim)*a₋(dim))
en=kron([0 0;0 1],I(dim))

ν=1.
    
function H(Ω,Δ)
    unitary::Matrix{ComplexF64}=kron(I(dim*2),-im*(ν*n-Δ*en+Ω/2*a2+Ω'/2*a3))+kron(transpose(im*(ν*n-Δ*en+Ω/2*a2+Ω'/2*a3)),I(dim*2))
    return unitary
end
    
function Hv(Ω,Δ)
    unitary::Matrix{ComplexF64}=ν*n-Δ*en+Ω/2*a2+Ω'/2*a3
    return unitary
end


Hv (generic function with 1 method)

In [3]:
instate=kron([1. 0;0 0],equ_state(dim,2));

In [4]:
function R(t,phi,omega,delta)
    return exp(-im*t*Hv(omega*exp(im*phi),delta))
end

R (generic function with 1 method)

In [5]:
function swap_up(dims,n)#下态到上态就行
    
    s=zeros(dims,dims)+I(dims)
    s[n+1,n+1]=0.;
   
    
    return s
end
swap=swap_up(dim,2)

4×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0

In [6]:
# function swap_prod(dims,n)#蓝边带交换
#     s=zeros(dim*2,dim*2)+I(dim*2)
#     s[n+1,n+1]=0.;
#     s[n+2+dims,n+2+dims]=0.;
#     s[n+2+dims,n+1]=1.;
#     s[n+1,n+2+dims]=1.;
    
    
#     return s
# end
# swap=swap_prod(dim,2)

In [7]:
function feval(x)
    swapfit=R(x[4],x[6],1,x[1])*R(x[3],x[5],1,x[1])*R(x[2],0,1,x[1])
    opres=norm(norm.(swapfit[1:dim,1:dim])-norm.(swap))
    println(round.(x,digits=5),"    ",round(opres,digits=4))    
    return opres
end

feval (generic function with 1 method)

In [18]:
xini=[0.73004, 149.59965, 140.97333, 145.15425, 6.28319, 1.46806]
# xini=res.minimizer
# xini=[0.93126, 148.1721, 149.50025, 140.63154, 1.11823, 6.37707]#feom0
# xin=[1.72755, 145.29433, 149.08461, 144.1136, 5.57726, 6.57151]#from1
# xini=[0.73346, 149.89149, 140.06108, 144.9223, 6.87165, 1.50956]#from2
lower=[0.,0,0,0,0,0];upper=[2.5,4*pi/η,4*pi/η,4*pi/η,2*pi,2*pi];
# res=optimize(feval,xini,ParticleSwarm(; lower = lower,upper = upper,n_particles = 10),Optim.Options(iterations = 10000))
res= optimize(feval,xini,LBFGS())   

[0.73005, 149.59965, 140.97333, 145.15425, 6.28319, 1.46806]    0.2655
[0.73003, 149.59965, 140.97333, 145.15425, 6.28319, 1.46806]    0.2655
[0.73004, 149.60056, 140.97333, 145.15425, 6.28319, 1.46806]    0.2655
[0.73004, 149.59874, 140.97333, 145.15425, 6.28319, 1.46806]    0.2655
[0.73004, 149.59965, 140.97418, 145.15425, 6.28319, 1.46806]    0.2655
[0.73004, 149.59965, 140.97248, 145.15425, 6.28319, 1.46806]    0.2655
[0.73004, 149.59965, 140.97333, 145.15513, 6.28319, 1.46806]    0.2655
[0.73004, 149.59965, 140.97333, 145.15337, 6.28319, 1.46806]    0.2655
[0.73004, 149.59965, 140.97333, 145.15425, 6.28323, 1.46806]    0.2655
[0.73004, 149.59965, 140.97333, 145.15425, 6.28315, 1.46806]    0.2655
[0.73004, 149.59965, 140.97333, 145.15425, 6.28319, 1.46807]    0.2655
[0.73004, 149.59965, 140.97333, 145.15425, 6.28319, 1.46805]    0.2655
[0.73004, 149.59965, 140.97333, 145.15425, 6.28319, 1.46806]    0.2655
[0.70638, 149.59953, 140.97321, 145.15416, 6.33799, 1.46814]    1.049
[0.7063

[0.72964, 149.60056, 140.97333, 145.15424, 6.44283, 1.46806]    0.2611
[0.72964, 149.59874, 140.97333, 145.15424, 6.44283, 1.46806]    0.2611
[0.72964, 149.59965, 140.97418, 145.15424, 6.44283, 1.46806]    0.2611
[0.72964, 149.59965, 140.97247, 145.15424, 6.44283, 1.46806]    0.2611
[0.72964, 149.59965, 140.97333, 145.15512, 6.44283, 1.46806]    0.2611
[0.72964, 149.59965, 140.97333, 145.15336, 6.44283, 1.46806]    0.2611
[0.72964, 149.59965, 140.97333, 145.15424, 6.44287, 1.46806]    0.2611
[0.72964, 149.59965, 140.97333, 145.15424, 6.44279, 1.46806]    0.2611
[0.72964, 149.59965, 140.97333, 145.15424, 6.44283, 1.46807]    0.2611
[0.72964, 149.59965, 140.97333, 145.15424, 6.44283, 1.46805]    0.2611
[0.72964, 149.59965, 140.97333, 145.15424, 6.44283, 1.46806]    0.2611
[0.72965, 149.59965, 140.97333, 145.15424, 6.43995, 1.46806]    0.2611
[0.72964, 149.59965, 140.97333, 145.15424, 6.43995, 1.46806]    0.2611
[0.72964, 149.60056, 140.97333, 145.15424, 6.43995, 1.46806]    0.2611
[0.729

[0.731, 149.63814, 140.8565, 144.9627, 6.68706, 1.65526]    0.2546
[0.731, 149.63814, 140.8565, 144.9627, 6.68706, 1.65524]    0.2546
[0.731, 149.63814, 140.8565, 144.9627, 6.68706, 1.65525]    0.2546
[0.73096, 149.63654, 140.86138, 144.9706, 6.67655, 1.64764]    0.2546
[0.73094, 149.63654, 140.86138, 144.9706, 6.67655, 1.64764]    0.2546
[0.73095, 149.63744, 140.86138, 144.9706, 6.67655, 1.64764]    0.2546
[0.73095, 149.63563, 140.86138, 144.9706, 6.67655, 1.64764]    0.2546
[0.73095, 149.63654, 140.86223, 144.9706, 6.67655, 1.64764]    0.2546
[0.73095, 149.63654, 140.86053, 144.9706, 6.67655, 1.64764]    0.2546
[0.73095, 149.63654, 140.86138, 144.97148, 6.67655, 1.64764]    0.2546
[0.73095, 149.63654, 140.86138, 144.96972, 6.67655, 1.64764]    0.2546
[0.73095, 149.63654, 140.86138, 144.9706, 6.67659, 1.64764]    0.2546
[0.73095, 149.63654, 140.86138, 144.9706, 6.67651, 1.64764]    0.2546
[0.73095, 149.63654, 140.86138, 144.9706, 6.67655, 1.64765]    0.2546
[0.73095, 149.63654, 140.86

[0.73231, 149.70218, 140.61174, 144.77016, 6.93923, 1.63446]    0.2491
[0.7323, 149.70218, 140.61174, 144.77016, 6.93923, 1.63446]    0.2492
[0.73231, 149.70309, 140.61174, 144.77016, 6.93923, 1.63446]    0.2491
[0.73231, 149.70128, 140.61174, 144.77016, 6.93923, 1.63446]    0.2492
[0.73231, 149.70218, 140.61259, 144.77016, 6.93923, 1.63446]    0.2492
[0.73231, 149.70218, 140.61089, 144.77016, 6.93923, 1.63446]    0.2491
[0.73231, 149.70218, 140.61174, 144.77104, 6.93923, 1.63446]    0.2491
[0.73231, 149.70218, 140.61174, 144.76928, 6.93923, 1.63446]    0.2492
[0.73231, 149.70218, 140.61174, 144.77016, 6.93927, 1.63446]    0.2491
[0.73231, 149.70218, 140.61174, 144.77016, 6.93919, 1.63446]    0.2491
[0.73231, 149.70218, 140.61174, 144.77016, 6.93923, 1.63447]    0.2491
[0.73231, 149.70218, 140.61174, 144.77016, 6.93923, 1.63445]    0.2491
[0.73231, 149.70218, 140.61174, 144.77016, 6.93923, 1.63446]    0.2491
[0.73226, 149.69965, 140.62121, 144.77825, 6.93188, 1.634]    0.2491
[0.73225,

[0.73255, 149.71302, 140.56104, 144.76093, 6.90017, 1.5697]    0.2476
[0.73255, 149.71302, 140.56104, 144.76093, 6.90017, 1.56971]    0.2476
[0.73258, 149.71398, 140.55752, 144.75777, 6.89892, 1.57037]    0.2476
[0.73257, 149.71398, 140.55752, 144.75777, 6.89892, 1.57037]    0.2476
[0.73258, 149.71489, 140.55752, 144.75777, 6.89892, 1.57037]    0.2476
[0.73258, 149.71308, 140.55752, 144.75777, 6.89892, 1.57037]    0.2476
[0.73258, 149.71398, 140.55837, 144.75777, 6.89892, 1.57037]    0.2476
[0.73258, 149.71398, 140.55667, 144.75777, 6.89892, 1.57037]    0.2475
[0.73258, 149.71398, 140.55752, 144.75865, 6.89892, 1.57037]    0.2475
[0.73258, 149.71398, 140.55752, 144.75689, 6.89892, 1.57037]    0.2476
[0.73258, 149.71398, 140.55752, 144.75777, 6.89896, 1.57037]    0.2476
[0.73258, 149.71398, 140.55752, 144.75777, 6.89888, 1.57037]    0.2476
[0.73258, 149.71398, 140.55752, 144.75777, 6.89892, 1.57038]    0.2476
[0.73258, 149.71398, 140.55752, 144.75777, 6.89892, 1.57036]    0.2476
[0.7325

[0.73357, 149.83249, 140.13147, 144.86132, 6.89658, 1.54012]    0.2361
[0.73357, 149.8334, 140.13147, 144.86132, 6.89658, 1.54012]    0.2361
[0.73357, 149.83158, 140.13147, 144.86132, 6.89658, 1.54012]    0.2361
[0.73357, 149.83249, 140.13232, 144.86132, 6.89658, 1.54012]    0.2361
[0.73357, 149.83249, 140.13062, 144.86132, 6.89658, 1.54012]    0.2361
[0.73357, 149.83249, 140.13147, 144.8622, 6.89658, 1.54012]    0.2361
[0.73357, 149.83249, 140.13147, 144.86044, 6.89658, 1.54012]    0.2361
[0.73357, 149.83249, 140.13147, 144.86132, 6.89662, 1.54012]    0.2361
[0.73357, 149.83249, 140.13147, 144.86132, 6.89653, 1.54012]    0.2361
[0.73357, 149.83249, 140.13147, 144.86132, 6.89658, 1.54013]    0.2361
[0.73357, 149.83249, 140.13147, 144.86132, 6.89658, 1.54011]    0.2361
[0.73357, 149.83249, 140.13147, 144.86132, 6.89658, 1.54012]    0.2361
[0.73771, 150.11523, 139.08236, 144.76665, 7.34347, 1.38836]    0.3844
[0.7377, 150.11523, 139.08236, 144.76665, 7.34347, 1.38836]    0.3847
[0.73771,

[0.73368, 149.85249, 140.05882, 144.9173, 6.89358, 1.51753]    0.2353
[0.73368, 149.85249, 140.05882, 144.9173, 6.89358, 1.51754]    0.2353
[0.73368, 149.85246, 140.05891, 144.91725, 6.89364, 1.51753]    0.2353
[0.73367, 149.85246, 140.05891, 144.91725, 6.89364, 1.51753]    0.2353
[0.73368, 149.85337, 140.05891, 144.91725, 6.89364, 1.51753]    0.2353
[0.73368, 149.85156, 140.05891, 144.91725, 6.89364, 1.51753]    0.2353
[0.73368, 149.85246, 140.05976, 144.91725, 6.89364, 1.51753]    0.2353
[0.73368, 149.85246, 140.05807, 144.91725, 6.89364, 1.51753]    0.2353
[0.73368, 149.85246, 140.05891, 144.91813, 6.89364, 1.51753]    0.2353
[0.73368, 149.85246, 140.05891, 144.91637, 6.89364, 1.51753]    0.2353
[0.73368, 149.85246, 140.05891, 144.91725, 6.89369, 1.51753]    0.2353
[0.73368, 149.85246, 140.05891, 144.91725, 6.8936, 1.51753]    0.2353
[0.73368, 149.85246, 140.05891, 144.91725, 6.89364, 1.51754]    0.2353
[0.73368, 149.85246, 140.05891, 144.91725, 6.89364, 1.51752]    0.2353
[0.73368,

[0.73329, 149.90475, 140.08842, 144.91537, 6.83634, 1.50268]    0.2353
[0.73329, 149.90475, 140.08842, 144.91624, 6.83638, 1.50268]    0.2353
[0.73329, 149.90475, 140.08842, 144.91624, 6.8363, 1.50268]    0.2353
[0.73329, 149.90475, 140.08842, 144.91624, 6.83634, 1.50268]    0.2353
[0.73329, 149.90475, 140.08842, 144.91624, 6.83634, 1.50267]    0.2353
[0.73329, 149.90475, 140.08842, 144.91624, 6.83634, 1.50268]    0.2353
[0.7335, 149.87764, 140.07346, 144.91662, 6.86617, 1.51033]    0.2352
[0.73348, 149.87764, 140.07346, 144.91662, 6.86617, 1.51033]    0.2352
[0.73349, 149.87855, 140.07346, 144.91662, 6.86617, 1.51033]    0.2352
[0.73349, 149.87673, 140.07346, 144.91662, 6.86617, 1.51033]    0.2352
[0.73349, 149.87764, 140.0743, 144.91662, 6.86617, 1.51033]    0.2352
[0.73349, 149.87764, 140.07261, 144.91662, 6.86617, 1.51033]    0.2352
[0.73349, 149.87764, 140.07346, 144.9175, 6.86617, 1.51033]    0.2352
[0.73349, 149.87764, 140.07346, 144.91574, 6.86617, 1.51033]    0.2352
[0.73349, 

[0.73346, 149.89149, 140.06108, 144.92317, 6.87165, 1.50956]    0.2352
[0.73346, 149.89149, 140.06108, 144.92142, 6.87165, 1.50956]    0.2352
[0.73346, 149.89149, 140.06108, 144.9223, 6.87169, 1.50956]    0.2352
[0.73346, 149.89149, 140.06108, 144.9223, 6.87161, 1.50956]    0.2352
[0.73346, 149.89149, 140.06108, 144.9223, 6.87165, 1.50957]    0.2352
[0.73346, 149.89149, 140.06108, 144.9223, 6.87165, 1.50955]    0.2352
[0.73346, 149.89149, 140.06108, 144.9223, 6.87165, 1.50956]    0.2352
[0.73346, 149.89149, 140.06108, 144.9223, 6.87165, 1.50956]    0.2352
[0.73345, 149.89149, 140.06108, 144.9223, 6.87165, 1.50956]    0.2352
[0.73346, 149.8924, 140.06108, 144.9223, 6.87165, 1.50956]    0.2352
[0.73346, 149.89058, 140.06108, 144.9223, 6.87165, 1.50956]    0.2352
[0.73346, 149.89149, 140.06193, 144.9223, 6.87165, 1.50956]    0.2352
[0.73346, 149.89149, 140.06023, 144.9223, 6.87165, 1.50956]    0.2352
[0.73346, 149.89149, 140.06108, 144.92317, 6.87165, 1.50956]    0.2352
[0.73346, 149.8914

 * Status: success (objective increased between iterations)

 * Candidate solution
    Final objective value:     2.351876e-01

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 1.34e-09 ≰ 0.0e+00
    |x - x'|/|x'|          = 8.93e-12 ≰ 0.0e+00
    |f(x) - f(x')|         = 9.71e-15 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 4.13e-14 ≰ 0.0e+00
    |g(x)|                 = 1.10e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    28
    f(x) calls:    102
    ∇f(x) calls:   102


In [10]:
# xini

In [11]:
res.minimizer

6-element Vector{Float64}:
   0.7334580523143855
 149.89149209735086
 140.0610774965967
 144.9222970038077
   6.871650099623819
   1.5095615703910685

In [12]:
res.minimum

0.23518758274592355

In [13]:
function feval2(x)
    swapfit=R(x[4],x[6],1,x[1])*R(x[3],x[5],1,x[1])*R(x[2],0,1,x[1])
    
    return swapfit
end

feval2 (generic function with 1 method)

In [14]:
round.(DataFrame(abs.(feval2(xini)),:auto),digits=4)

Row,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.9185,0.1495,0.0051,0.0009,0.3487,0.1109,0.0084,0.0025
2,0.1331,0.9786,0.0439,0.0049,0.0422,0.1251,0.0701,0.0156
3,0.0069,0.0587,0.0698,0.0802,0.002,0.0491,0.8415,0.524
4,0.001,0.0062,0.0783,0.9808,0.0006,0.0053,0.1008,0.1472
5,0.3467,0.0252,0.0062,0.0009,0.925,0.1534,0.0044,0.0015
6,0.1353,0.1111,0.0531,0.0073,0.145,0.9713,0.0419,0.0159
7,0.0084,0.0575,0.8697,0.144,0.0054,0.0461,0.295,0.3609
8,0.0029,0.0114,0.4772,0.1039,0.002,0.0227,0.4334,0.757


In [15]:
# round.(DataFrame(feval2(xini)/(feval2(xini)[1]/abs.(feval2(xini))[1]),:auto),digits=4)

In [16]:
round.(DataFrame(abs.(feval2(res.minimizer)),:auto),digits=4)

Row,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.964,0.1436,0.0095,0.0005,0.168,0.146,0.0199,0.0024
2,0.1382,0.9659,0.032,0.007,0.0297,0.1918,0.0931,0.0222
3,0.0096,0.0712,0.046,0.0154,0.0046,0.047,0.886,0.4531
4,0.001,0.0127,0.0588,0.9708,0.0006,0.0061,0.1094,0.2048
5,0.1689,0.0033,0.0014,0.0003,0.9848,0.0405,0.0053,0.0008
6,0.1513,0.1896,0.0584,0.0098,0.0332,0.9671,0.0272,0.0214
7,0.0033,0.0662,0.8712,0.1614,0.0013,0.0448,0.2407,0.3881
8,0.0021,0.0286,0.4806,0.1765,0.0018,0.0257,0.3678,0.7754
